In [1]:
import numpy as np
from seisLM.data_pipeline.foreshock_aftershock_dataset import (
  create_foreshock_aftershock_datasets
)
from seisLM.data_pipeline.foreshock_aftershock_dataloaders import (
  prepare_foreshock_aftershock_dataloaders)
import pandas as pd
import matplotlib.pyplot as plt
from seisLM.utils.project_path import gitdir

/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# from seisLM.model.foundation.pretrained_models import LitMultiDimWav2Vec2

# model = LitMultiDimWav2Vec2.load_from_checkpoint(
#   "/home/liu0003/Desktop/projects/seisLM/results/models/pretrained_seisLM/pretrain_config_layernorm_std_small_batch_4_datasets_less_regularized_42__2024-08-07-16h-54m-15s/checkpoints/epoch=24-step=437425.ckpt",
# )#.model



/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.3.3, which is newer than your current Lightning version: v2.2.5


In [5]:
# model.config

data_config:
  amp_norm: true
  amp_norm_type: std
  cache_dataset: null
  data_name:
  - ETHZ
  - GEOFON
  - STEAD
  - InstanceCounts
  demean: true
  local_batch_size: 28
  num_workers: 4
  prefetch_factor: 2
  training_fraction: 1.0
model_config:
  activation_dropout: 0.0
  apply_spec_augment: true
  attention_dropout: 0.0
  codevector_dim: 256
  contrastive_logits_temperature: 0.1
  conv_bias: true
  conv_dim:
  - 256
  - 256
  conv_embed: true
  conv_kernel:
  - 3
  - 3
  conv_stride:
  - 2
  - 2
  diversity_loss_weight: 0.01
  do_stable_layer_norm: true
  feat_extract_norm: layer
  feat_proj_dropout: 0.0
  feat_quantizer_dropout: 0.0
  hidden_dropout: 0.0
  hidden_size: 240
  initializer_range: 0.02
  input_dim: 3
  intermediate_size: 3072
  layer_norm_eps: 1.0e-05
  layerdrop: 0.0
  mask_feature_length: 10
  mask_feature_min_masks: 0
  mask_feature_prob: 0.0
  mask_time_length: 10
  mask_time_min_masks: 2
  mask_time_prob: 0.65
  num_attention_heads: 12
  num_codevector_groups: 

: 

In [2]:
import json
import ml_collections
import torch
from seisLM.model.task_specific.foreshock_aftershock_models import (
  Conv1DShockClassifierLit, Wav2vec2ShockClassifierLit)


NUM_CLASSES = 4

def get_seisLM_config():

  config_path = gitdir() + '/seisLM/configs/foreshock_aftershock/seisLM_shock_classifier.json'
  with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
  config = ml_collections.ConfigDict(config)

  config.model_args.num_classes = NUM_CLASSES
  return config


def get_conv1d_config():
  config_path = gitdir() + '/seisLM/configs/foreshock_aftershock/conv1d_shock_classifier.json'
  with open(config_path, "r", encoding="utf-8") as f:
    config = json.load(f)
  config = ml_collections.ConfigDict(config)
  config.model_args.num_classes = NUM_CLASSES
  return config



seisLM_config = get_seisLM_config()
conv1d_config = get_conv1d_config()

models = {}

models['seisLM'] = Wav2vec2ShockClassifierLit(
  model_config=seisLM_config.model_args,
  training_config=seisLM_config.trainer_args,
  ).model


models['cnn'] = Conv1DShockClassifierLit(
  model_config=conv1d_config.model_args,
  training_config=conv1d_config.trainer_args,
  ).model


loaders = prepare_foreshock_aftershock_dataloaders(
  num_classes=seisLM_config.model_args.num_classes,
  **seisLM_config.data_args
)


/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/lightning/pytorch/utilities/migration/utils.py:56: The loaded checkpoint was produced with Lightning v2.3.3, which is newer than your current Lightning version: v2.2.5
Seed set to 42
Seed set to 42


In [3]:
from sklearn.metrics import accuracy_score


for key, model in models.items():
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model = models[key].to(device)

  all_gt_class = []
  all_pred_class = []
  all_losses = []

  for X, y in loaders['test']:
    with torch.no_grad():
      logits = model(X.to(device)).cpu()
      pred_class = np.argmax(logits.numpy(), axis=1)
      loss = torch.nn.functional.cross_entropy(logits, y)

    all_gt_class.append(y)
    all_pred_class.append(pred_class)
    all_losses.append(loss)

  all_gt_class = np.concatenate(all_gt_class)
  all_pred_class = np.concatenate(all_pred_class)
  all_losses = np.array(all_losses)

  print(f'model {key} acc at init', accuracy_score(all_gt_class, all_pred_class))
  print(f'model {key} loss at init', np.mean(all_losses))

model seisLM acc at init 0.27348066298342544
model seisLM loss at init 1.4177186
model cnn acc at init 0.25
model cnn loss at init 1.3920208


: 